In [142]:
import os
import glob
import datetime
import numpy as np
import pandas as pd

pd.set_option('max_rows', 100)
pd.set_option('max_columns', 100)

perc = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]

startDate = '20200911'
endDate = '20200911'

sl = ['SZ123015', 'SZ123032', 'SZ128070', 'SZ123042', 'SZ123034',
       'SZ123048', 'SZ123061', 'SH113576', 'SZ123065', 'SZ123030',
       'SZ123045', 'SZ123013', 'SZ127004', 'SZ128103', 'SZ128073',
       'SZ128041', 'SZ123012', 'SZ123062', 'SH110044', 'SZ128079',
       'SZ123052', 'SZ128036', 'SZ123014', 'SZ128091', 'SZ123018',
       'SZ123031', 'SZ123038', 'SH113035', 'SZ128052', 'SZ128053',
       'SZ128039', 'SZ128050', 'SZ123023', 'SH113555', 'SZ127008',
       'SZ123047', 'SZ123051', 'SZ123024', 'SZ123029', 'SZ128076',
       'SZ123026', 'SZ123027', 'SZ128112', 'SH113521', 'SZ123057',
       'SZ128108', 'SH113520', 'SZ123046', 'SH113038', 'SH113011']
sl1 = [i[2:] for i in sl if i[:2] == 'SZ']
sl2 = [i[2:] for i in sl if i[:2] == 'SH']

readPath = r'A:\KR_daily_data\***'
dataPathLs = np.array(glob.glob(readPath))
dateLs = np.array([os.path.basename(i) for i in dataPathLs])
dateLs = dateLs[(dateLs >= startDate) & (dateLs <= endDate)]
for ii in dateLs:
    order = []
    trade = []
    snapshot = []
    os.mkdir('G:\\convertible_bond\\' + ii)
    for j in sl1:
        readPath = 'A:\\KR_daily_data\\' + ii + '\\SZ\\order\\' + j + '.csv'
        dataPathLs = np.array(glob.glob(readPath))
        try:
            orderLog = pd.read_csv(dataPathLs[0])
            orderLog['SecurityID'] = int(j)
        except:
            print(j)
            continue
        
        orderLog = orderLog.rename(columns={"OrdType": "OrderType"})
        orderLog["date"] = orderLog["TransactTime"].iloc[0]//1000000000
        orderLog["OrderType"] = np.where(orderLog["OrderType"] == 'U', 3, orderLog["OrderType"])
        orderLog["skey"] = orderLog["SecurityID"] + 2000000
        orderLog["clockAtArrival"] = orderLog["TransactTime"].astype(str).apply(lambda x: np.int64(datetime.datetime.strptime(x, '%Y%m%d%H%M%S%f').timestamp()*1e6))
        orderLog['datetime'] = orderLog["clockAtArrival"].apply(lambda x: datetime.datetime.fromtimestamp(x/1e6))
        orderLog["time"] = (orderLog['TransactTime'] - int(orderLog['TransactTime'].iloc[0]//1000000000*1000000000)).astype(np.int64)*1000

        for col in ["skey", "date", "ApplSeqNum", "OrderQty", "Side", "OrderType"]:
            orderLog[col] = orderLog[col].astype('int32')

        orderLog = orderLog.rename(columns={"Side":"order_side", "OrderType":"order_type", "Price":"order_price", "OrderQty":"order_qty"})
        orderLog = orderLog[["skey", "date", "time", "clockAtArrival", "datetime", "ApplSeqNum", "order_side", "order_type", "order_price",
                                                     "order_qty"]]
        
            
        readPath = 'A:\\KR_daily_data\\' + ii + '\\SZ\\tick\\' + j + '.csv'
        dataPathLs = np.array(glob.glob(readPath))
        try:
            tradeLog = pd.read_csv(dataPathLs[0])
            tradeLog['SecurityID'] = int(j)
        except:
            continue
            
        tradeLog["date"] = tradeLog["TransactTime"].iloc[0]//1000000000
        tradeLog = tradeLog.rename(columns={"Qty":"trade_qty", "Price":"trade_price", "ExecType":"trade_type"})
        tradeLog["trade_money"] = tradeLog["trade_price"] * tradeLog["trade_qty"]
        tradeLog["trade_flag"] = 0
        tradeLog["skey"] = tradeLog["SecurityID"] + 2000000
        tradeLog["clockAtArrival"] = tradeLog["TransactTime"].astype(str).apply(lambda x: np.int64(datetime.datetime.strptime(x, '%Y%m%d%H%M%S%f').timestamp()*1e6))
        tradeLog['datetime'] = tradeLog["clockAtArrival"].apply(lambda x: datetime.datetime.fromtimestamp(x/1e6))
        tradeLog["time"] = (tradeLog['TransactTime'] - int(tradeLog['TransactTime'].iloc[0]//1000000000*1000000000)).astype(np.int64)*1000
        tradeLog["trade_type"] = np.where(tradeLog["trade_type"] == 'F', 1, tradeLog["trade_type"])
        for col in ["skey", "date", "ApplSeqNum", "BidApplSeqNum", "OfferApplSeqNum", "trade_qty", "trade_type", "trade_flag"]:
            tradeLog[col] = tradeLog[col].astype('int32')
        tradeLog = tradeLog[["skey", "date", "time", "clockAtArrival", "datetime", "ApplSeqNum", "trade_type", "trade_flag",
                                                     "trade_price", "trade_qty", "BidApplSeqNum", "OfferApplSeqNum"]]
        
        
        readPath = 'A:\\KR_daily_data\\' + ii + '\\SZ\\snapshot\\Level2\\' + j + '.csv'
        dataPathLs = np.array(glob.glob(readPath))
        try:
            SZ = pd.read_csv(dataPathLs[0], usecols = [0,1,4,5,6,7,9,12,17,18,19,24,25,26,28,29,30,32,33,34,35])
            SZ['StockID'] = int(j)
        except:
            continue
        
        SZ["skey"] = SZ["StockID"] + 2000000
        SZ.drop(["StockID"],axis=1,inplace=True)
        SZ["date"] = int(SZ["QuotTime"].iloc[0]//1000000000)
        SZ["time"] = (SZ['QuotTime'] - int(SZ['QuotTime'].iloc[0]//1000000000*1000000000)).astype(np.int64) * 1000
        SZ["clockAtArrival"] = SZ["QuotTime"].astype(str).apply(lambda x: np.int64(datetime.datetime.strptime(x, '%Y%m%d%H%M%S%f').timestamp()*1e6))
        SZ['datetime'] = SZ["clockAtArrival"].apply(lambda x: datetime.datetime.fromtimestamp(x/1e6))
        SZ.drop(["QuotTime"],axis=1,inplace=True)

        SZ["BidPrice"] = SZ["BidPrice"].apply(lambda x: [float(i) for i in x[1:-1].split(',')])
        SZ["OfferPrice"] = SZ["OfferPrice"].apply(lambda x: [float(i) for i in x[1:-1].split(',')])
        SZ["BidOrderQty"] = SZ["BidOrderQty"].apply(lambda x: [int(i) for i in x[1:-1].split(',')])
        SZ["OfferOrderQty"] = SZ["OfferOrderQty"].apply(lambda x: [int(i) for i in x[1:-1].split(',')])
        SZ["BidNumOrders"] = SZ["BidNumOrders"].apply(lambda x: [int(i) for i in x[1:-1].split(',')])
        SZ["OfferNumOrders"] = SZ["OfferNumOrders"].apply(lambda x: [int(i) for i in x[1:-1].split(',')])

        for i in range(1, 11):
            SZ["bid" + str(i) + 'p'] = SZ["BidPrice"].apply(lambda x: x[i-1],2)
        SZ.drop(["BidPrice"],axis=1,inplace=True)
        for i in range(1, 11):
            SZ["ask" + str(i) + 'p'] = SZ["OfferPrice"].apply(lambda x: x[i-1],2)
        SZ.drop(["OfferPrice"],axis=1,inplace=True)
        for i in range(1, 11):
            SZ["bid" + str(i) + 'q'] = SZ["BidOrderQty"].apply(lambda x: x[i-1])
        SZ.drop(["BidOrderQty"],axis=1,inplace=True)
        for i in range(1, 11):
            SZ["ask" + str(i) + 'q'] = SZ["OfferOrderQty"].apply(lambda x: x[i-1])
        SZ.drop(["OfferOrderQty"],axis=1,inplace=True)
        for i in range(1, 11):
            SZ["bid" + str(i) + 'n'] = SZ["BidNumOrders"].apply(lambda x: x[i-1])
            SZ["bid" + str(i) + 'n'] = SZ["bid" + str(i) + 'n'].astype('int32')
        SZ.drop(["BidNumOrders"],axis=1,inplace=True)
        for i in range(1, 11):
            SZ["ask" + str(i) + 'n'] = SZ["OfferNumOrders"].apply(lambda x: x[i-1])
            SZ["ask" + str(i) + 'n'] = SZ["ask" + str(i) + 'n'].astype('int32') 
        SZ.drop(["OfferNumOrders"],axis=1,inplace=True)

        SZ["BidOrders"] = SZ["BidOrders"].apply(lambda x: [int(i) for i in x[1:-1].split(',')])
        SZ["OfferOrders"] = SZ["OfferOrders"].apply(lambda x: [int(i) for i in x[1:-1].split(',')])

        for i in range(1, 51):
            SZ["bid1Top" + str(i) + 'q'] = SZ["BidOrders"].apply(lambda x: x[i-1])
            SZ["bid1Top" + str(i) + 'q'] = SZ["bid1Top" + str(i) + 'q'].astype('int32') 
        SZ.drop(["BidOrders"],axis=1,inplace=True)

        for i in range(1, 51):
            SZ["ask1Top" + str(i) + 'q'] = SZ["OfferOrders"].apply(lambda x: x[i-1])
            SZ["ask1Top" + str(i) + 'q'] = SZ["ask1Top" + str(i) + 'q'].astype('int32') 
        SZ.drop(["OfferOrders"],axis=1,inplace=True)


        SZ.columns = ['cum_trades_cnt', 'total_bid_quantity', 'total_ask_quantity', 'close',
           'total_ask_vwap', 'cum_amount', 'cum_volume', 'open', 'high',
           'prev_close', 'low', 'total_bid_vwap', 'skey', 'date', 'time',
           'clockAtArrival', 'datetime', 'bid1p', 'bid2p', 'bid3p', 'bid4p',
           'bid5p', 'bid6p', 'bid7p', 'bid8p', 'bid9p', 'bid10p', 'ask1p',
           'ask2p', 'ask3p', 'ask4p', 'ask5p', 'ask6p', 'ask7p', 'ask8p',
           'ask9p', 'ask10p', 'bid1q', 'bid2q', 'bid3q', 'bid4q', 'bid5q',
           'bid6q', 'bid7q', 'bid8q', 'bid9q', 'bid10q', 'ask1q', 'ask2q',
           'ask3q', 'ask4q', 'ask5q', 'ask6q', 'ask7q', 'ask8q', 'ask9q',
           'ask10q', 'bid1n', 'bid2n', 'bid3n', 'bid4n', 'bid5n', 'bid6n',
           'bid7n', 'bid8n', 'bid9n', 'bid10n', 'ask1n', 'ask2n', 'ask3n',
           'ask4n', 'ask5n', 'ask6n', 'ask7n', 'ask8n', 'ask9n', 'ask10n',
           'bid1Top1q', 'bid1Top2q', 'bid1Top3q', 'bid1Top4q', 'bid1Top5q',
           'bid1Top6q', 'bid1Top7q', 'bid1Top8q', 'bid1Top9q', 'bid1Top10q',
           'bid1Top11q', 'bid1Top12q', 'bid1Top13q', 'bid1Top14q',
           'bid1Top15q', 'bid1Top16q', 'bid1Top17q', 'bid1Top18q',
           'bid1Top19q', 'bid1Top20q', 'bid1Top21q', 'bid1Top22q',
           'bid1Top23q', 'bid1Top24q', 'bid1Top25q', 'bid1Top26q',
           'bid1Top27q', 'bid1Top28q', 'bid1Top29q', 'bid1Top30q',
           'bid1Top31q', 'bid1Top32q', 'bid1Top33q', 'bid1Top34q',
           'bid1Top35q', 'bid1Top36q', 'bid1Top37q', 'bid1Top38q',
           'bid1Top39q', 'bid1Top40q', 'bid1Top41q', 'bid1Top42q',
           'bid1Top43q', 'bid1Top44q', 'bid1Top45q', 'bid1Top46q',
           'bid1Top47q', 'bid1Top48q', 'bid1Top49q', 'bid1Top50q',
           'ask1Top1q', 'ask1Top2q', 'ask1Top3q', 'ask1Top4q', 'ask1Top5q',
           'ask1Top6q', 'ask1Top7q', 'ask1Top8q', 'ask1Top9q', 'ask1Top10q',
           'ask1Top11q', 'ask1Top12q', 'ask1Top13q', 'ask1Top14q',
           'ask1Top15q', 'ask1Top16q', 'ask1Top17q', 'ask1Top18q',
           'ask1Top19q', 'ask1Top20q', 'ask1Top21q', 'ask1Top22q',
           'ask1Top23q', 'ask1Top24q', 'ask1Top25q', 'ask1Top26q',
           'ask1Top27q', 'ask1Top28q', 'ask1Top29q', 'ask1Top30q',
           'ask1Top31q', 'ask1Top32q', 'ask1Top33q', 'ask1Top34q',
           'ask1Top35q', 'ask1Top36q', 'ask1Top37q', 'ask1Top38q',
           'ask1Top39q', 'ask1Top40q', 'ask1Top41q', 'ask1Top42q',
           'ask1Top43q', 'ask1Top44q', 'ask1Top45q', 'ask1Top46q',
           'ask1Top47q', 'ask1Top48q', 'ask1Top49q', 'ask1Top50q']

        SZ = SZ.fillna(0)
        SZ["ordering"] = SZ.groupby("skey").cumcount()
        SZ["ordering"] = SZ["ordering"] + 1

        for cols in ["total_bid_orders",'total_ask_orders','total_bid_levels', 'total_ask_levels', 'bid_trade_max_duration',
                     'ask_trade_max_duration', 'cum_canceled_buy_orders', 'cum_canceled_buy_volume', "cum_canceled_buy_amount",
                     "cum_canceled_sell_orders", 'cum_canceled_sell_volume',"cum_canceled_sell_amount", "has_missing"]:
            SZ[cols] = 0
            
        for col in ["skey", "date", "cum_trades_cnt", "total_bid_orders",
            'total_ask_orders', 'total_bid_levels', 'total_ask_levels', 'cum_canceled_buy_orders','cum_canceled_sell_orders',
                "ordering", 'bid_trade_max_duration', 'ask_trade_max_duration','has_missing']:
            SZ[col] = SZ[col].astype('int32')

        for cols in ["cum_canceled_sell_amount", "cum_canceled_buy_amount"]:
            SZ[cols] = SZ[cols].astype('float64')


        assert(sum(SZ[SZ["open"] != 0].groupby("skey")["open"].nunique() != 1) == 0)
        assert(sum(SZ[SZ["prev_close"] != 0].groupby("skey")["prev_close"].nunique() != 1) == 0)
        SZ["prev_close"] = np.where(SZ["time"] >= 91500000000, SZ.groupby("skey")["prev_close"].transform("max"), SZ["prev_close"]) 
        SZ["open"] = np.where(SZ["cum_volume"] > 0, SZ.groupby("skey")["open"].transform("max"), SZ["open"])
        assert(sum(SZ[SZ["open"] != 0].groupby("skey")["open"].nunique() != 1) == 0)
        assert(sum(SZ[SZ["prev_close"] != 0].groupby("skey")["prev_close"].nunique() != 1) == 0)
        assert(SZ[SZ["cum_volume"] > 0]["open"].min() > 0)   
        
        SZ["has_missing"] = SZ["has_missing"].astype('int32')
        SZ = SZ[["skey", "date", "time", "clockAtArrival", "datetime", "ordering", "has_missing", "cum_trades_cnt", "cum_volume", "cum_amount", "prev_close",
                                "open", "high", "low", "close", 'bid10p','bid9p','bid8p','bid7p','bid6p','bid5p','bid4p','bid3p','bid2p','bid1p',
                                'ask1p','ask2p','ask3p','ask4p','ask5p','ask6p','ask7p','ask8p','ask9p','ask10p', 'bid10q','bid9q','bid8q',
                                 'bid7q','bid6q','bid5q','bid4q','bid3q','bid2q','bid1q', 'ask1q','ask2q','ask3q','ask4q','ask5q','ask6q',
                                 'ask7q','ask8q','ask9q','ask10q', 'bid10n', 'bid9n', 'bid8n', 'bid7n', 'bid6n', 'bid5n', 'bid4n', 'bid3n', 'bid2n', 'bid1n', 
                                 'ask1n', 'ask2n', 'ask3n', 'ask4n', 'ask5n', 'ask6n','ask7n', 'ask8n', 'ask9n', 'ask10n','bid1Top1q','bid1Top2q','bid1Top3q','bid1Top4q','bid1Top5q','bid1Top6q',
            'bid1Top7q','bid1Top8q','bid1Top9q','bid1Top10q','bid1Top11q','bid1Top12q','bid1Top13q','bid1Top14q','bid1Top15q','bid1Top16q','bid1Top17q','bid1Top18q',
            'bid1Top19q','bid1Top20q','bid1Top21q','bid1Top22q','bid1Top23q','bid1Top24q','bid1Top25q','bid1Top26q','bid1Top27q','bid1Top28q','bid1Top29q',
            'bid1Top30q','bid1Top31q','bid1Top32q','bid1Top33q','bid1Top34q','bid1Top35q','bid1Top36q','bid1Top37q','bid1Top38q','bid1Top39q','bid1Top40q',
            'bid1Top41q','bid1Top42q','bid1Top43q','bid1Top44q','bid1Top45q','bid1Top46q','bid1Top47q','bid1Top48q','bid1Top49q','bid1Top50q', 'ask1Top1q',
            'ask1Top2q','ask1Top3q','ask1Top4q','ask1Top5q','ask1Top6q','ask1Top7q','ask1Top8q','ask1Top9q','ask1Top10q','ask1Top11q','ask1Top12q','ask1Top13q',
            'ask1Top14q','ask1Top15q','ask1Top16q','ask1Top17q','ask1Top18q','ask1Top19q','ask1Top20q','ask1Top21q','ask1Top22q','ask1Top23q',
            'ask1Top24q','ask1Top25q','ask1Top26q','ask1Top27q','ask1Top28q','ask1Top29q','ask1Top30q','ask1Top31q','ask1Top32q','ask1Top33q',
            'ask1Top34q','ask1Top35q','ask1Top36q','ask1Top37q','ask1Top38q','ask1Top39q','ask1Top40q','ask1Top41q','ask1Top42q','ask1Top43q',
            'ask1Top44q','ask1Top45q','ask1Top46q','ask1Top47q','ask1Top48q','ask1Top49q','ask1Top50q',"total_bid_quantity", "total_ask_quantity","total_bid_vwap", "total_ask_vwap",
            "total_bid_orders",'total_ask_orders','total_bid_levels', 'total_ask_levels', 'bid_trade_max_duration', 'ask_trade_max_duration', 'cum_canceled_buy_orders', 'cum_canceled_buy_volume',
            "cum_canceled_buy_amount", "cum_canceled_sell_orders", 'cum_canceled_sell_volume',"cum_canceled_sell_amount"]]
        snapshot += [SZ]
        order += [orderLog]
        trade += [tradeLog]
    
    for j in sl2:
        readPath = 'A:\\KR_daily_data\\' + ii + '\\SH\\tick\\' + j + '.csv'
        dataPathLs = np.array(glob.glob(readPath))
        try:
            tradeLog = pd.read_csv(dataPathLs[0])
            tradeLog['SecurityID'] = int(j)
        except:
            print(j)
            continue
        
        tradeLog["date"] = tradeLog["TradeTime"].iloc[0]//1000000000
        tradeLog = tradeLog.rename(columns={"TradeQty":"trade_qty", "TradePrice":"trade_price", 
                                            "TradeBSFlag":"trade_flag", "TradeAmount":"trade_money",
                                           "TradeIndex":"ApplSeqNum", "SellNo":"OfferApplSeqNum",
                                           "BuyNo":"BidApplSeqNum"})
        tradeLog["trade_type"] = 1
        tradeLog["skey"] = tradeLog["SecurityID"] + 1000000
        tradeLog["clockAtArrival"] = tradeLog["TradeTime"].astype(str).apply(lambda x: np.int64(datetime.datetime.strptime(x, '%Y%m%d%H%M%S%f').timestamp()*1e6))
        tradeLog['datetime'] = tradeLog["clockAtArrival"].apply(lambda x: datetime.datetime.fromtimestamp(x/1e6))
        tradeLog["time"] = (tradeLog['TradeTime'] - int(tradeLog['TradeTime'].iloc[0]//1000000000*1000000000)).astype(np.int64)*1000
        tradeLog["trade_flag"] = np.where(tradeLog["trade_flag"] == 'B', 1, np.where(
            tradeLog["trade_flag"] == 'S', 2, 0))
        for col in ["skey", "date", "ApplSeqNum", "BidApplSeqNum", "OfferApplSeqNum", "trade_qty", "trade_type", "trade_flag"]:
            tradeLog[col] = tradeLog[col].astype('int32')

        tradeLog = tradeLog[["skey", "date", "time", "clockAtArrival", "datetime", "ApplSeqNum", "trade_type", "trade_flag",
                                                     "trade_price", "trade_qty", "BidApplSeqNum", "OfferApplSeqNum"]]
        
        
        readPath = 'A:\\KR_daily_data\\' + ii + '\\SH\\snapshot\\Level2\\' + j + '.csv'
        dataPathLs = np.array(glob.glob(readPath))
        try:
            SH = pd.read_csv(dataPathLs[0], usecols = [0,1,3,5,7,9,10,11,15,17,18,19,20,21,22,23,25,26,28,29,30,31,32,33,37,39,40,41,
                                          42,46,47,49,50])
            SH['StockID'] = int(j)
        except:
            print(j)
            continue        

        SH["skey"] = SH["StockID"] + 1000000
        SH.drop(["StockID"],axis=1,inplace=True)
        SH["date"] = int(SH["QuotTime"].iloc[0]//1000000000)
        SH["time"] = (SH['QuotTime'] - int(SH['QuotTime'].iloc[0]//1000000000*1000000000)).astype(np.int64) * 1000
        SH["clockAtArrival"] = SH["QuotTime"].astype(str).apply(lambda x: np.int64(datetime.datetime.strptime(x, '%Y%m%d%H%M%S%f').timestamp()*1e6))
        SH.drop(["QuotTime"],axis=1,inplace=True)
        SH['datetime'] = SH["clockAtArrival"].apply(lambda x: datetime.datetime.fromtimestamp(x/1e6))

        SH["BidPrice"] = SH["BidPrice"].apply(lambda x: [float(i) for i in x[1:-1].split(',')])
        SH["OfferPrice"] = SH["OfferPrice"].apply(lambda x: [float(i) for i in x[1:-1].split(',')])
        SH["BidOrderQty"] = SH["BidOrderQty"].apply(lambda x: [int(i) for i in x[1:-1].split(',')])
        SH["OfferOrderQty"] = SH["OfferOrderQty"].apply(lambda x: [int(i) for i in x[1:-1].split(',')])
        SH["BidNumOrders"] = SH["BidNumOrders"].apply(lambda x: [int(i) for i in x[1:-1].split(',')])
        SH["OfferNumOrders"] = SH["OfferNumOrders"].apply(lambda x: [int(i) for i in x[1:-1].split(',')])

        for i in range(1, 11):
            SH["bid" + str(i) + 'p'] = SH["BidPrice"].apply(lambda x: x[i-1],2)
        SH.drop(["BidPrice"],axis=1,inplace=True)

        for i in range(1, 11):
            SH["ask" + str(i) + 'p'] = SH["OfferPrice"].apply(lambda x: x[i-1],2)
        SH.drop(["OfferPrice"],axis=1,inplace=True)

        for i in range(1, 11):
            SH["bid" + str(i) + 'q'] = SH["BidOrderQty"].apply(lambda x: x[i-1])
        SH.drop(["BidOrderQty"],axis=1,inplace=True)

        for i in range(1, 11):
            SH["ask" + str(i) + 'q'] = SH["OfferOrderQty"].apply(lambda x: x[i-1])
        SH.drop(["OfferOrderQty"],axis=1,inplace=True)

        for i in range(1, 11):
            SH["bid" + str(i) + 'n'] = SH["BidNumOrders"].apply(lambda x: x[i-1])
            SH["bid" + str(i) + 'n'] = SH["bid" + str(i) + 'n'].astype('int32')
        SH.drop(["BidNumOrders"],axis=1,inplace=True)

        for i in range(1, 11):
            SH["ask" + str(i) + 'n'] = SH["OfferNumOrders"].apply(lambda x: x[i-1])
            SH["ask" + str(i) + 'n'] = SH["ask" + str(i) + 'n'].astype('int32') 
        SH.drop(["OfferNumOrders"],axis=1,inplace=True)


        SH["BidOrders"] = SH["BidOrders"].apply(lambda x: [int(i) for i in x[1:-1].split(',')])
        SH["OfferOrders"] = SH["OfferOrders"].apply(lambda x: [int(i) for i in x[1:-1].split(',')])

        for i in range(1, 51):
            SH["bid1Top" + str(i) + 'q'] = SH["BidOrders"].apply(lambda x: x[i-1])
            SH["bid1Top" + str(i) + 'q'] = SH["bid1Top" + str(i) + 'q'].astype('int32') 
        SH.drop(["BidOrders"],axis=1,inplace=True)


        for i in range(1, 51):
            SH["ask1Top" + str(i) + 'q'] = SH["OfferOrders"].apply(lambda x: x[i-1])
            SH["ask1Top" + str(i) + 'q'] = SH["ask1Top" + str(i) + 'q'].astype('int32') 
        SH.drop(["OfferOrders"],axis=1,inplace=True)

        SH.columns = ['cum_trades_cnt', 'ask_trade_max_duration', 'total_bid_orders',
           'cum_canceled_sell_amount', 'total_ask_quantity', 'cum_canceled_buy_orders',
           'total_ask_vwap', 'cum_canceled_sell_volume', 'cum_volume', 'open',
           'high', 'prev_close', 'low', 'total_bid_vwap',
           'cum_canceled_sell_orders', 'total_ask_orders', 'total_ask_levels',
           'total_bid_quantity', 'cum_canceled_buy_volume', 'bid_trade_max_duration',
           'total_bid_levels', 'close', 'cum_amount', 'cum_canceled_buy_amount', 'skey', 'date', 'time', 'clockAtArrival',
           'datetime', 'bid1p', 'bid2p', 'bid3p', 'bid4p', 'bid5p', 'bid6p',
           'bid7p', 'bid8p', 'bid9p', 'bid10p', 'ask1p', 'ask2p', 'ask3p',
           'ask4p', 'ask5p', 'ask6p', 'ask7p', 'ask8p', 'ask9p', 'ask10p',
           'bid1q', 'bid2q', 'bid3q', 'bid4q', 'bid5q', 'bid6q', 'bid7q',
           'bid8q', 'bid9q', 'bid10q', 'ask1q', 'ask2q', 'ask3q', 'ask4q',
           'ask5q', 'ask6q', 'ask7q', 'ask8q', 'ask9q', 'ask10q', 'bid1n',
           'bid2n', 'bid3n', 'bid4n', 'bid5n', 'bid6n', 'bid7n', 'bid8n',
           'bid9n', 'bid10n', 'ask1n', 'ask2n', 'ask3n', 'ask4n', 'ask5n',
           'ask6n', 'ask7n', 'ask8n', 'ask9n', 'ask10n', 'bid1Top1q',
           'bid1Top2q', 'bid1Top3q', 'bid1Top4q', 'bid1Top5q', 'bid1Top6q',
           'bid1Top7q', 'bid1Top8q', 'bid1Top9q', 'bid1Top10q', 'bid1Top11q',
           'bid1Top12q', 'bid1Top13q', 'bid1Top14q', 'bid1Top15q',
           'bid1Top16q', 'bid1Top17q', 'bid1Top18q', 'bid1Top19q',
           'bid1Top20q', 'bid1Top21q', 'bid1Top22q', 'bid1Top23q',
           'bid1Top24q', 'bid1Top25q', 'bid1Top26q', 'bid1Top27q',
           'bid1Top28q', 'bid1Top29q', 'bid1Top30q', 'bid1Top31q',
           'bid1Top32q', 'bid1Top33q', 'bid1Top34q', 'bid1Top35q',
           'bid1Top36q', 'bid1Top37q', 'bid1Top38q', 'bid1Top39q',
           'bid1Top40q', 'bid1Top41q', 'bid1Top42q', 'bid1Top43q',
           'bid1Top44q', 'bid1Top45q', 'bid1Top46q', 'bid1Top47q',
           'bid1Top48q', 'bid1Top49q', 'bid1Top50q', 'ask1Top1q', 'ask1Top2q',
           'ask1Top3q', 'ask1Top4q', 'ask1Top5q', 'ask1Top6q', 'ask1Top7q',
           'ask1Top8q', 'ask1Top9q', 'ask1Top10q', 'ask1Top11q', 'ask1Top12q',
           'ask1Top13q', 'ask1Top14q', 'ask1Top15q', 'ask1Top16q',
           'ask1Top17q', 'ask1Top18q', 'ask1Top19q', 'ask1Top20q',
           'ask1Top21q', 'ask1Top22q', 'ask1Top23q', 'ask1Top24q',
           'ask1Top25q', 'ask1Top26q', 'ask1Top27q', 'ask1Top28q',
           'ask1Top29q', 'ask1Top30q', 'ask1Top31q', 'ask1Top32q',
           'ask1Top33q', 'ask1Top34q', 'ask1Top35q', 'ask1Top36q',
           'ask1Top37q', 'ask1Top38q', 'ask1Top39q', 'ask1Top40q',
           'ask1Top41q', 'ask1Top42q', 'ask1Top43q', 'ask1Top44q',
           'ask1Top45q', 'ask1Top46q', 'ask1Top47q', 'ask1Top48q',
           'ask1Top49q', 'ask1Top50q']
        SH = SH.fillna(0)
        SH["ordering"] = SH.groupby("skey").cumcount()
        SH["ordering"] = SH["ordering"] + 1

        SH["has_missing"] = 0

        for col in ["skey", "date", "cum_trades_cnt", "total_bid_orders",
            'total_ask_orders', 'total_bid_levels', 'total_ask_levels', 'cum_canceled_buy_orders','cum_canceled_sell_orders',
                "ordering", 'bid_trade_max_duration', 'ask_trade_max_duration','has_missing']:
            SH[col] = SH[col].astype('int32')

        assert(sum(SH[SH["open"] != 0].groupby("skey")["open"].nunique() != 1) == 0)
        assert(sum(SH[SH["prev_close"] != 0].groupby("skey")["prev_close"].nunique() != 1) == 0)
        SH["prev_close"] = np.where(SH["time"] >= 91500000000, SH.groupby("skey")["prev_close"].transform("max"), SH["prev_close"]) 
        SH["open"] = np.where(SH["cum_volume"] > 0, SH.groupby("skey")["open"].transform("max"), SH["open"])
        assert(sum(SH[SH["open"] != 0].groupby("skey")["open"].nunique() != 1) == 0)
        assert(sum(SH[SH["prev_close"] != 0].groupby("skey")["prev_close"].nunique() != 1) == 0)
        assert(SH[SH["cum_volume"] > 0]["open"].min() > 0)

        SH["has_missing"] = SH["has_missing"].astype('int32')
        SH = SH[["skey", "date", "time", "clockAtArrival", "datetime", "ordering", "has_missing", "cum_trades_cnt", "cum_volume", "cum_amount", "prev_close",
                                "open", "high", "low", "close", 'bid10p','bid9p','bid8p','bid7p','bid6p','bid5p','bid4p','bid3p','bid2p','bid1p',
                                'ask1p','ask2p','ask3p','ask4p','ask5p','ask6p','ask7p','ask8p','ask9p','ask10p', 'bid10q','bid9q','bid8q',
                                 'bid7q','bid6q','bid5q','bid4q','bid3q','bid2q','bid1q', 'ask1q','ask2q','ask3q','ask4q','ask5q','ask6q',
                                 'ask7q','ask8q','ask9q','ask10q', 'bid10n', 'bid9n', 'bid8n', 'bid7n', 'bid6n', 'bid5n', 'bid4n', 'bid3n', 'bid2n', 'bid1n', 
                                 'ask1n', 'ask2n', 'ask3n', 'ask4n', 'ask5n', 'ask6n','ask7n', 'ask8n', 'ask9n', 'ask10n','bid1Top1q','bid1Top2q','bid1Top3q','bid1Top4q','bid1Top5q','bid1Top6q',
            'bid1Top7q','bid1Top8q','bid1Top9q','bid1Top10q','bid1Top11q','bid1Top12q','bid1Top13q','bid1Top14q','bid1Top15q','bid1Top16q','bid1Top17q','bid1Top18q',
            'bid1Top19q','bid1Top20q','bid1Top21q','bid1Top22q','bid1Top23q','bid1Top24q','bid1Top25q','bid1Top26q','bid1Top27q','bid1Top28q','bid1Top29q',
            'bid1Top30q','bid1Top31q','bid1Top32q','bid1Top33q','bid1Top34q','bid1Top35q','bid1Top36q','bid1Top37q','bid1Top38q','bid1Top39q','bid1Top40q',
            'bid1Top41q','bid1Top42q','bid1Top43q','bid1Top44q','bid1Top45q','bid1Top46q','bid1Top47q','bid1Top48q','bid1Top49q','bid1Top50q', 'ask1Top1q',
            'ask1Top2q','ask1Top3q','ask1Top4q','ask1Top5q','ask1Top6q','ask1Top7q','ask1Top8q','ask1Top9q','ask1Top10q','ask1Top11q','ask1Top12q','ask1Top13q',
            'ask1Top14q','ask1Top15q','ask1Top16q','ask1Top17q','ask1Top18q','ask1Top19q','ask1Top20q','ask1Top21q','ask1Top22q','ask1Top23q',
            'ask1Top24q','ask1Top25q','ask1Top26q','ask1Top27q','ask1Top28q','ask1Top29q','ask1Top30q','ask1Top31q','ask1Top32q','ask1Top33q',
            'ask1Top34q','ask1Top35q','ask1Top36q','ask1Top37q','ask1Top38q','ask1Top39q','ask1Top40q','ask1Top41q','ask1Top42q','ask1Top43q',
            'ask1Top44q','ask1Top45q','ask1Top46q','ask1Top47q','ask1Top48q','ask1Top49q','ask1Top50q',"total_bid_quantity", "total_ask_quantity","total_bid_vwap", "total_ask_vwap",
            "total_bid_orders",'total_ask_orders','total_bid_levels', 'total_ask_levels', 'bid_trade_max_duration', 'ask_trade_max_duration', 'cum_canceled_buy_orders', 'cum_canceled_buy_volume',
            "cum_canceled_buy_amount", "cum_canceled_sell_orders", 'cum_canceled_sell_volume',"cum_canceled_sell_amount"]]   
        
        snapshot += [SH]
        trade += [tradeLog]
    
    snapshot = pd.concat(snapshot).reset_index(drop=True)
    trade = pd.concat(trade).reset_index(drop=True)
    order = pd.concat(order).reset_index(drop=True)
    pd.to_pickle(snapshot, 'G:\\convertible_bond\\' + ii + '\\snapshot.pkl')
    pd.to_pickle(order, 'G:\\convertible_bond\\' + ii + '\\order.pkl')
    pd.to_pickle(trade, 'G:\\convertible_bond\\' + ii + '\\trade.pkl')
    print(ii + ' done')

        
    

123065
123052
20200911 done


In [138]:
dateLs[22:]

array(['20201009', '20201012', '20201013', '20201014', '20201015',
       '20201016', '20201019', '20201020', '20201021', '20201022',
       '20201023', '20201026', '20201027', '20201028', '20201029'],
      dtype='<U8')

In [136]:
j

'113576'